## Audiobooks business case

### Problem
You are given data from an Audiobook App. Logically, it relates to the audio versions of books ONLY. Each customer in the database has made a purchase at least once, that's why he/she is in the database. We want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.\
The main idea is that if a customer has a low probability of coming back, there is no reason to spend any money on advertising to him/her. If we can focus our efforts SOLELY on customers that are likely to convert again, we can make great savings. Moreover, this model can identify the most important metrics for a customer to come back again. Identifying new customers creates value and growth opportunities.
We have a .csv summarizing the data. There are several variables: Customer ID, ), Book length overall (sum of the minute length of all purchases), Book length avg (average length in minutes of all purchases), Price paid_overall (sum of all purchases) ,Price Paid avg (average of all purchases), Review (a Boolean variable whether the customer left a review), Review out of 10 (if the customer left a review, his/her review out of 10, Total minutes listened, Completion (from 0 to 1), Support requests (number of support requests; everything from forgotten password to assistance for using the App), and Last visited minus purchase date (in days).
These are the inputs (excluding customer ID, as it is completely arbitrary. It's more like a name, than a number).
The targets are a Boolean variable (0 or 1). We are taking a period of 2 years in our inputs, and the next 6 months as targets. So, in fact, we are predicting if: based on the last 2 years of activity and engagement, a customer will convert in the next 6 months. 6 months sounds like a reasonable time. If they don't convert after 6 months, chances are they've gone to a competitor or didn't like the Audiobook way of digesting information.
The task is simple: create a machine learning algorithm, which is able to predict if a customer will buy again.
This is a classification problem with two classes: won't buy and will buy, represented by 0s and 1s.

### Preprocess the data. Balance the dataset. Create 3 datasets: training, validation, and test. Save the newly created sets in a tensor friendly format (e.g. *.npz)¶
Since we are dealing with real life data, we will need to preprocess it a bit. This is the relevant code, which is not that hard, but is crucial to creating a good model.
Note that we have removed the header row, which contains the names of the categories. We simply want the data.

### Import Libraries 

In [1]:
import numpy as np 
from sklearn import preprocessing
import os 

In [2]:
# using os to get file path
file_path = os.path.join(os.path.expanduser("~"), "Desktop", "The Data Science Course 2020 - All Resources/Part_7_Deep_Learning/S51_L357", "Audiobooks_data.csv")
# load data with np.loadtxt(filepath,delemiter)
raw_csv_data = np.loadtxt(file_path, delimiter=",")
# The inputs are all columns in csv file, except for the first one bacause it it ID 
# (which is just the arbitrary customer IDs that bear no useful information)
# And we need to except the last columns, because it is target
unscaled_input_all = raw_csv_data[:,1:-1]
# The target is the last columns. That's how datasets are conventionally organized.
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [3]:
# Count how many targets are one (meaning that customer ID converted)
num_ones_targets = int(np.sum(targets_all))
# Set the counter for targets is 0 (meaning that customer ID didn't convert)
zero_targets_counter = 0 
# If we want to "balanced" dataset, we will have to remove some input/target pairs.
# Declare an empty list to prepare reserve indices to remove
indices_to_remove = []

for i in range(targets_all.shape[0]) :
    if targets_all[i] == 0 :
        zero_targets_counter += 1 
        if zero_targets_counter > num_ones_targets :
            indices_to_remove.append(i)
# Create two new variables, one will contain inputs, the other will contain targets
# Additionally, we need to remove indices which marked in indices_to_remove list
unscaled_input_all_priors = np.delete(unscaled_input_all, indices_to_remove, axis = 0)
targets_all_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standarlize the inputs

In [4]:
# Here is the only place we use sklearn. We will take advantage of its preprocessing capabilities
# It's a simple line code, which standardizes inputs
scaled_input_all = preprocessing.scale(unscaled_input_all_priors)

### Shuffle data

In [5]:
# When the data was collected, it was actually arranged by date
# Shuffle the indices of data, so the data is not arranged in any way when we feed it
# Since we will be batching,  we need the data to be as randomly spread out as possible
shuffled_indice = np.arange(scaled_input_all.shape[0])
# use np.random.shuffle() to shuffle inputs
np.random.shuffle(shuffled_indice)
# use shuffle_indice to shuffle inputs and targets 
shuffled_inputs= scaled_input_all[shuffled_indice]
shuffled_targets = targets_all_priors[shuffled_indice]

### Split the dataset into train, validation, and test

In [6]:
# Count the total number of samples
num_samples_data = shuffled_inputs.shape[0]
# Set the size of sample for each subsets, assuming train_data = 80%, validation_data = 10%, test_data = 10%
train_samples_count = int(0.8*num_samples_data)
valid_samples_count = int(0.1*num_samples_data)
test_samples_count = num_samples_data - train_samples_count - valid_samples_count
# Create variables to record the inputs and target for training data
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]
# Create variables to record the inputs and target for validation data 
validation_inputs = shuffled_inputs[train_samples_count : train_samples_count + valid_samples_count ]
validation_targets = shuffled_targets[train_samples_count : train_samples_count + valid_samples_count ]
# Create variables to record the inputs and target for test data 
test_inputs = shuffled_inputs[train_samples_count + valid_samples_count : ]
test_targets = shuffled_targets[train_samples_count + valid_samples_count : ]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.

In [8]:
print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count )
print(np.sum(validation_targets), valid_samples_count, np.sum(validation_targets)/valid_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/ test_samples_count)

1786.0 3579 0.4990220732048058
227.0 447 0.5078299776286354
224.0 448 0.5


### Save the three datasets in *.npz

In [11]:
# if you want to save at expected folder, you should set destination directory
destination_dir = os.path.join(os.path.expanduser("~"), "Desktop")
np.savez(os.path.join(destination_dir,"Audiobooks_data_train"), inputs=train_inputs, targets=train_targets)
np.savez(os.path.join(destination_dir,"Audiobooks_validation_train"), inputs=validation_inputs, targets=validation_targets)
np.savez(os.path.join(destination_dir,"Audiobooks_test_train"), inputs=test_inputs, targets=test_targets)